## Import stuff

In [1]:
# Jupyter shenanigans
%matplotlib widget
%config InlineBackend.figure_format='retina'

# Science 
import numpy as np
from scipy import interpolate, ndimage, signal

# Plotting and visualization
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Data tools
import h5py
import sqlite3

# MongoDB
from customImports import mongo 
client = mongo.client_connect()
db = mongo.database_connect(client,'Experiments')
collection = db.Thermography

# Misc
from datetime import datetime, timedelta
import importlib
import copy

# Set base directory for macOS
import os.path
dir = '/'.join(os.getcwd().split('/')[0:3])+'/'
print(dir)

/Users/howiechu/


## Import files

In [2]:
# Set file directory, experiment folder always contains A.mat for matlab array
# and exptname.sqlite or .csv (Ivium vs. kepco) for voltage data
thermdir = dir+'Desktop/Kokam/KKMG/SOH100/SOC50/KKMG_SOC50_5c100s_230320_2/'
ecf = 'KKMG_SOC50_5c100s_230320_2.csv'

In [3]:
# load IR video array
def reloadvid():
    A = h5py.File(thermdir+'A.mat','r')[('A')]
    # Transpose Array (some older sets are transposed for some reason)
    if A.shape[1]>A.shape[2]:
        A = np.transpose(A,(0,2,1))
    return A

A = reloadvid()

## Importing and syncing timestamps

In [4]:
# Load IR camera time stamps
At = np.loadtxt(thermdir+'timestamps.csv', 
               delimiter=',', dtype=str)
At = [datetime.strptime(time[4:],'%H:%M:%S.%f') for time in At]
dstA = At[0]
At = np.asarray([datetime.timestamp(n) for n in At])

# Load Echem data from DAQ
EC = np.loadtxt(thermdir+ecf,
               delimiter=',', 
               skiprows=9, 
               usecols=np.arange(1,4))
ECt = np.loadtxt(thermdir+ecf, 
               delimiter=',', skiprows=9, usecols=0, dtype=str)
dstE = (datetime.strptime(ECt[0][1:-3],'%H:%M:%S.%f'))

### Check for DST (DAQ doesn't use OS time)

In [5]:
print('dstA = ' + str(dstA))
print('dstE = ' + str(dstE))

if dstA>dstE:
    dstCheck = dstA-dstE
else:
    dstCheck = dstE-dstA

print('dstCheck = ' + str(dstCheck))

if dstCheck.seconds > 600:
    ECt = [datetime.strptime(time[1:-3],'%H:%M:%S.%f')-timedelta(hours=1) for time in ECt]
else:
    ECt = [datetime.strptime(time[1:-3],'%H:%M:%S.%f') for time in ECt]
ECt = np.asarray([datetime.timestamp(n) for n in ECt])

dstA = 1900-01-01 18:08:07.459000
dstE = 1900-01-01 18:08:04.703771
dstCheck = 0:00:02.755229


### Find when current starts to flow

In [6]:
# Finds where absolute value of current is greater than 0.2 the first time
off = np.argmax(np.diff(np.where(np.absolute(EC[:,1])<0.2)))
timeOffset = ECt[off+1]
print('timeOffset =' +str(timeOffset))

# Time sync : 0 is now when current starts flowing
At = At-timeOffset
ECt = ECt-timeOffset

# Put time into the DAQ array
EC = np.insert(EC,0,ECt,axis=1)

timeOffset =-2208923498.896229


### Set thermometer temperature to calibrate ambient thermocouple

In [7]:
tref = 23.5
EC[:,1] = EC[:,1] - EC[0,1] + tref

### Create kokam object

In [8]:
from customImports import thermography_functions as tf
# Reload module if changes are made
# importlib.reload(tf)
kokam = tf.thermography(A, 'Kokam')

### Plots DAQ data, change col to get other column data

In [10]:
kokam.plot_DAQ(EC, ylabel = 'Ambient temperature ($^{\circ}$C)', col = [1])
# kokam.plot_DAQ(EC, ylabel = 'Potential (V)', col = 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Adjust ROI locations (full image)
Manually change parameters until rectangles align, then uncomment last line to save figure
- Sampling area : ndarray[row_index][column_index]
- With patches  : patches.thingyouwant((x,y),width,height,etc)

In [11]:
# Manually choose where the active battery material and ambient spot locations are
# Placed into list [top left corner row, top left corner col, number of rows, number of columns] or [y, x, height, width]
# Can probably do something like edge detection instead
Abatt = [139,102,52,109]
Aamb = [139,277,27,27]

## Plots the ROI that are chosen above

In [12]:
kokam.plot_ROI(Abatt, Aamb, frame = 2000)
# Save current figure, if wanted
# There is also a function to save all open figures near the end
# plt.savefig(thermdir+"ROI_return.pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Slice out ROI data for further processing

In [ ]:
B, Amb = kokam.extract_ROI(Abatt, Aamb)
# Not necessary but might need to reload the original file if working backwards
B = copy.deepcopy(B)

### Camera calibration adjustment

In [ ]:
B = kokam.ambient_calibration(B, Amb)

### Hot spot tracker

In [ ]:
# Finds index of IR frame that references  t = 0
At0 = np.where(np.round(At) == 0)[0][0]
# Function to generate list of 2D max locations in (x,y) for n frames after t = 0
indMax = kokam.hotspot_location_array(B, At0, 2500)

In [ ]:
# Average the x and y coordinate 
# (patchSize/2) is for making up for the dimension of the square
patchSize = 10
Ahot = np.round(np.mean(indMax[200:], axis=0)).astype(int)-int(patchSize/2)
# Acold is just defaulted to bottom left corner, shifted to match patchSize
Acold = [B[0].shape[0]-patchSize, 0]

print('Hot spot location (pixels): '+str(Ahot))
print('Cold spot location (pixels): '+str(Acold))

### Plots location of hot spot at each time step using magma color map
- dark/blue = early
- orange/white = later

In [ ]:
kokam.plot_extremes_tracker(B[0].shape, indMax, Ahot, Acold, patchSize)

### Plots the hot/cold spot rectangles over a sample frame from IR video

In [ ]:
kokam.plot_battery_ROI(B, patchSize, 1000, Ahot, Acold)

# ROI data generation 
- takes surface average of the entire input array, and the slices provided

In [ ]:
Thot, Tavg, Tcold = kokam.extract_data(B, Ahot, Acold, patchSize)

#### Convert $\Delta$T back to absolute using average ambient temperature 
#### (from ambient temperature thermocouple calibrated with thermometer)

In [ ]:
# Add back the time average air temperature of the room (over time of experiment)
tAmbAvg = np.mean(EC[:,1])

Thot = Thot + tAmbAvg
Tavg = Tavg + tAmbAvg
Tcold = Tcold + tAmbAvg

# Combines the column vectors into an array
IR = np.stack((At,Thot,Tavg,Tcold), axis = 1)

# Interpolate all the things
- Camera freq is at 1 Hz
- Data logger is at 5 Hz (older experiments were at 1 Hz)
- linearly interpolate at 1 Hz

In [ ]:
Time = np.arange(0,int(At[-1]),1)
Table = np.empty((Time.shape[0],IR.shape[1]+1))
Table[:,0] = Time
for x in range(1,4):
    Table[:,x] = np.interp(Time,IR[:,0],IR[:,x])
Table[:,-1] = np.interp(Time,EC[:,0],EC[:,3])

### Normalize starting temperatures
- shifts every column so that t=0 has the same temperatures for hot/cold/average

In [ ]:
# Small adjustment to the starting temperatures
avgstart = np.mean(Table[0,1:4])
for x in range(1,4):
    Table[:,x] = Table[:,x]-Table[0,x]+avgstart
# np.savetxt(dir+expt+"thermographyData.csv", Table, delimiter=",", fmt='%1.3f')

# Plot

In [ ]:
kokam.plot_processed_thermography(Table, endTime = 2500, adjustable = True)

### Run these cells if there are bad calibrations in the data (jumps)

In [ ]:
# Adjust which series to analyze and peak height to choose most appropriate fix
kokam.analyze_jumps(Table, 3, 0.1)

In [ ]:
fTable = kokam.execute_jumps(Table)

In [ ]:
kokam.plot_processed_thermography(fTable, endTime = 2500, adjustable = True, figName = 'postFix')

In [ ]:
# np.savetxt(dir+expt+"thermographyData.csv", fTable, delimiter=",", fmt='%1.3f')

# Smoothing for video

In [ ]:
# Associate At (timestamps) to the B matrix, 
# then interp1D to get interpolated temp data at relevant times
Ainterp = interpolate.interp1d(At, B, axis=0)

# Time is the same dimensions as COMSOL data length
evalTime = np.arange(0,int(At[-1]),1)

# Evaluate interpolation function
Binterp = Ainterp(evalTime)

# Convert back to absolute temperature using tAmbAvg from previous cells
Binterp = Binterp - Binterp[0] + tAmbAvg

# Smooth the matrix for visualization (gaussian blur)
Bsmooth = ndimage.filters.gaussian_filter(Binterp, 5)

In [ ]:
# Use list of time steps you want to plot
plotTimes = [1000, 1500, 2100, 2400]
# Input number of columns you want the figure to have
kokam.plot_video_frames(Bsmooth, plotTimes, 2)

### Code to save all open figures into a single PDF

In [ ]:
def save_allfigs(filename, figs=None):
    '''
    Saves all open figures to a single pdf file if no figure labels are passed, 
    otherwise saves whatever strings are passed.
    
    Parameters:
        filename (str): filename for pdf
        figs (list of str): list of figure labels
        
    Returns:
        pdf file with a figure on each page
    '''
    with PdfPages('{}.pdf'.format(filename)) as pdf:
        if figs is None:
            figs = plt.get_figlabels()
        for fig in figs:
            tempFig = plt.figure(fig)
            pdf.savefig(tempFig, dpi = 300)

In [ ]:
save_allfigs('allfigs')

In [ ]:

doc_update = {
    'directory':thermdir,
    'filename': {
        'IR': thermdir.split('/')[-2],
        'EC': ecf
    },
    'ROI': {
        'battery corner':Abatt[0:2],
        'battery height':Abatt[2],
        'battery width':Abatt[3],
        'ambient corner':Aamb[0:2],
        'ambient height':Aamb[2],
        'ambient width':Aamb[3]
    },
    'expt':{
        'cell':'Kokam',
        'cellID':thermdir.split('/')[-2].split('_')[0],
        'type':'squarewave',
        'SOH':95,
        'SOC':60,
        'c-rate':2,
        'period':100,
        'tref':tref,
        'PSU':'octostat'
    },
    'batteryROI': {
        'hot corner':Ahot.tolist(),
        'hot height':patchSize,
        'hot width':patchSize,
        'cold corner':Acold,
        'cold height':patchSize,
        'cold width':patchSize
    },
    'output':mongo.convert_ndarray_binary(fTable),
    'imageLocations': {
        'thermogramFull':thermdir+"ROI_return.pdf",
        'hotSpotMap':thermdir+"hotSpotTracked.pdf",
        'thermogramBatt':thermdir+"battery_ROI.pdf",
        'thermogramTime':thermdir+'thermogramTime.pdf',
        'outputPlot':thermdir+'dataPlot.pdf'
    },
    'notes': 'nothing of note',
    'dstCheck':dstCheck if ecf.split('.')[-1] in ['csv'] else None,
    'timeOffset':timeOffset
    
}

collection.update_one({'filename.IR':thermdir.split('/')[-2]}, {'$set':doc_update}, upsert=True)